<a href="https://colab.research.google.com/github/FlyAIBox/AIAgent101/blob/main/06-agent-evaluation/langfuse/evaluation_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
在 Langfuse 上运行 LangChain 评测

本指南演示如何使用基于模型的评测，自动化评估 Langfuse 中线上产出的 LLM 完成结果。示例使用 LangChain，亦可迁移到其他库；最佳选择取决于具体场景。

本指南分三步：
1. 从 Langfuse 获取线上存储的 `generations`
2. 使用 LangChain 对这些 `generations` 进行评测
3. 将结果作为 `scores` 回灌到 Langfuse


----
还未使用 Langfuse？通过采集 LLM 事件，[立即开始](https://langfuse.com/docs/get-started)。

### 环境准备

先用 pip 安装 Langfuse 与 LangChain，然后设置环境变量。

In [ ]:
# 🧰 安装示例所需的核心依赖包
# 使用 IPython 的 %pip 魔法命令可以在 notebook 内直接安装依赖，效果等同于在终端执行 `pip install`
# - langfuse: Langfuse 平台的 Python SDK，用于记录与评测 LLM 应用
# - langchain: 构建大模型应用的主框架，提供链、代理、工具等抽象
# - langchain-openai: LangChain 对 OpenAI 系列模型的封装，便于统一调用接口
%pip install langfuse==3.3.0 langchain==0.3.27 langchain-openai==0.3.31


In [ ]:
# 🔐 环境变量配置：将敏感凭据安全地注入运行环境
import os
import getpass


def _set_env(var: str):
    """提示用户输入缺失的环境变量，避免把密钥硬编码到笔记本中。"""
    # 当环境变量尚未配置时，提示用户在控制台输入；getpass 会隐藏输入内容。
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# 🗝️ OpenAI API 配置 —— 评测阶段需要调用一个评估用的 LLM
# 请在 https://platform.openai.com/api-keys 申请密钥后，在此输入。
_set_env("OPENAI_API_KEY")

# 🌐 OpenAI 代理地址（可选）—— 如果你所在网络无法直接访问 OpenAI，可在此填写代理网关
# 常见格式如 https://api.xxx.com/v1；若无需代理可直接回车跳过。
_set_env("OPENAI_BASE_URL")

# 🛰️ Langfuse 平台配置 —— 用于托管与回放评测数据
# 所有密钥均可在 https://cloud.langfuse.com 的项目设置页面找到。
_set_env("LANGFUSE_PUBLIC_KEY")   # 公钥：标识项目身份
_set_env("LANGFUSE_SECRET_KEY")   # 私钥：用于身份认证，请勿泄露
_set_env("LANGFUSE_HOST")         # 服务地址：EU 区域 https://cloud.langfuse.com；US 区域 https://us.cloud.langfuse.com


LANGFUSE_PUBLIC_KEY: ··········
LANGFUSE_SECRET_KEY: ··········
LANGFUSE_HOST: ··········


In [ ]:
import os

# ⚙️ 指定在评测阶段要使用的 LLM 名称
# 这里默认采用 "gpt-3.5-turbo-instruct"，你也可以按需切换为 gpt-4 等更强模型（成本更高）。
os.environ["EVAL_MODEL"] = "gpt-3.5-turbo-instruct"

# 🗂️ 配置 LangChain 内置的多维度评测开关
# 将要启用的评测维度设置为 True；False 表示跳过该指标。
EVAL_TYPES = {
    "hallucination": True,   # 幻觉：输出是否包含输入或参考中不存在的虚假信息
    "conciseness": True,     # 简洁性：回答是否言简意赅、避免冗余
    "relevance": True,       # 相关性：回答是否紧扣提问或任务
    "coherence": True,       # 连贯性：段落组织是否顺畅、逻辑是否自洽
    "harmfulness": True,     # 有害性：是否出现危险、伤害或不当内容
    "maliciousness": True,   # 恶意性：是否有恶意意图，例如煽动攻击
    "helpfulness": True,     # 有用性：回答是否提供实质性帮助
    "controversiality": True,# 争议性：是否包含易引发争议或极端观点
    "misogyny": True,        # 性别歧视：是否具有歧视女性的言论
    "criminality": True,     # 犯罪性：是否鼓励或描述犯罪行为
    "insensitivity": True    # 不敏感性：是否对敏感群体、事件缺乏尊重
}


初始化 Langfuse Python SDK，更多信息见[此处](https://langfuse.com/docs/sdk/python#1-installation)。

In [ ]:
# 📡 连接 Langfuse 平台以读取评测样本
from langfuse import get_client

# Langfuse 客户端会自动读取刚才设置的环境变量完成认证
langfuse = get_client()

# ✅ 快速健康检查：确保密钥与网络配置正确
if langfuse.auth_check():
    print("Langfuse 客户端已通过认证，准备就绪！")
    print("现在可以开始从 Langfuse 获取数据并进行评测")
else:
    print("认证失败。请检查以下设置：")
    print("- LANGFUSE_PUBLIC_KEY / LANGFUSE_SECRET_KEY 是否填写正确")
    print("- LANGFUSE_HOST 是否指向正确的区域 (EU / US)")
    print("- 当前网络是否可以访问对应的 Langfuse 服务")


Langfuse 客户端已通过认证，准备就绪！


### 拉取数据

根据 `name` 从 Langfuse 载入所有 `generations`，此处示例为 `OpenAI`。在 Langfuse 中，`name` 用于标识应用内不同类型的生成。将其替换为你需要评测的名称。

关于在写入 LLM Generation 时如何设置 `name`，参见[文档](https://langfuse.com/docs/sdk/python#generation)。

In [ ]:
def fetch_all_pages(name=None, user_id=None, limit=50):
    """从 Langfuse 分页拉取 trace 数据，直到拿齐所有结果。"""
    page = 1            # Langfuse API 的页码从 1 起步
    all_data = []       # 用列表收集每一页返回的数据

    while True:
        # 通过 SDK 调用后端接口。可以附加 name / user_id 过滤条件，limit 控制单页大小。
        response = langfuse.api.trace.list(name=name, limit=limit, user_id=user_id, page=page)

        # 当某一页没有数据时，说明遍历完毕，跳出循环。
        if not response.data:
            break

        # 将当前页的所有 trace 追加到结果列表中
        all_data.extend(response.data)
        page += 1  # 自增页码，继续请求下一页

    return all_data


In [ ]:
# 📥 调用自定义工具，拉取指定用户的全部 trace
# 这里以 user_id='user_123' 为例，实际使用时请替换为你自己业务里记录的用户标识。
generations = fetch_all_pages(user_id='user_123')

print(f"成功获取到 {len(generations)} 条 trace 数据")


In [ ]:
# 🔎 快速浏览拉取到的原始数据结构，帮助理解后续字段的来源
def _print_generations_preview(items):
    if not items:
        print()  # 分隔提示信息
        print("⚠️ 没有找到任何 trace 数据！请检查下列事项：")
        print("1. user_id 是否填写正确")
        print("2. Langfuse 项目中是否已有生成记录")
        print("3. 当前网络能否访问 Langfuse")
        return

    print("获取到的 trace 数据示例 (仅展示前 3 条)：")
    for item in items[:3]:
        print("-" * 60)
        print(f"trace_id: {item.id}")
        print(f"input: {item.input}")
        print(f"output: {item.output}")
        print(f"timestamp: {item.timestamp}")

_print_generations_preview(generations)


[]


In [9]:
# 🆔 示例：查看第一条 trace 的唯一 ID，可在 Langfuse 前端用它定位记录
# 仅当成功拉取到数据后再访问列表元素，避免 IndexError。
if generations:
    generations[0].id


IndexError: list index out of range

### 定义评测函数

本节基于 `EVAL_TYPES` 定义 LangChain 评测器；其中“幻觉”（hallucination）需要单独函数。关于 LangChain 评测的更多信息见[此处](https://python.langchain.com/docs/guides/evaluation/string/criteria_eval_chain)。

In [ ]:
# 🛠️ 导入 LangChain 评测工具与 OpenAI 模型封装
from langchain.evaluation import load_evaluator
from langchain.evaluation.criteria import LabeledCriteriaEvalChain
from langchain_openai import OpenAI


def get_evaluator_for_key(key: str):
    """为指定的评测维度加载 LangChain 内置评测器。"""
    # temperature 设为 0 以获得确定性更高的评测结果。
    llm = OpenAI(temperature=0, model=os.environ.get("EVAL_MODEL"))
    # load_evaluator 会返回一个可直接调用的评测链对象。
    return load_evaluator("criteria", criteria=key, llm=llm)


def get_hallucination_eval():
    """单独构建“幻觉”维度的评测链（Hallucination 需要参考文本）。"""
    criteria = {
        "hallucination": (
            "Does this submission contain information not present in the input or reference?"
        )
    }
    llm = OpenAI(temperature=0, model=os.environ.get("EVAL_MODEL"))
    return LabeledCriteriaEvalChain.from_llm(llm=llm, criteria=criteria)


### 执行评测

下面将对上面载入的每个 `Generation` 执行评测。每个得分将通过 [`langfuse.score()`](https://langfuse.com/docs/scores) 写回 Langfuse。


In [ ]:
def execute_eval_and_score():
    """遍历所有 trace，针对开启的评测维度逐项打分。"""

    for generation in generations:
        # 过滤出所有开启的评测维度（除 hallucination 外，后者单独处理）
        criteria = [key for key, enabled in EVAL_TYPES.items() if enabled and key != "hallucination"]

        for criterion in criteria:
            # evaluate_strings 会返回一个包含 score 与 reasoning 的字典
            eval_result = get_evaluator_for_key(criterion).evaluate_strings(
                prediction=generation.output,
                input=generation.input,
            )
            print(eval_result)

            # 将评测得分写回 Langfuse，trace_id / observation_id 可用于后续回放
            langfuse.create_score(
                name=criterion,
                trace_id=generation.id,
                observation_id=generation.id,
                value=eval_result["score"],
                comment=eval_result["reasoning"],
            )


execute_eval_and_score()


In [ ]:
# 🎯 幻觉（hallucination）评测需要额外传入参考文本，这里单独处理

def eval_hallucination():
    chain = get_hallucination_eval()

    for generation in generations:
        eval_result = chain.evaluate_strings(
            prediction=generation.output,
            input=generation.input,
            reference=generation.input,  # 简单示例：以原始输入作为参考文本
        )
        print(eval_result)

        if (
            eval_result is not None
            and eval_result.get("score") is not None
            and eval_result.get("reasoning") is not None
        ):
            langfuse.create_score(
                name="hallucination",
                trace_id=generation.id,
                observation_id=generation.id,
                value=eval_result["score"],
                comment=eval_result["reasoning"],
            )


In [ ]:
# ✅ 根据配置决定是否执行幻觉评测
if EVAL_TYPES.get("hallucination"):
    eval_hallucination()


In [ ]:
# 📤 Langfuse Python SDK 内部使用异步队列发送数据，这里手动 flush 以确保所有打分已写入服务端
langfuse.flush()


### 在 Langfuse 中查看分数

在 Langfuse 界面中，你可以按 `Scores` 过滤 Traces，并查看每条的详细信息。你也可以通过 Langfuse Analytics 分析新提示词版本或应用发布对这些分数的影响。

![Trace 示例](https://langfuse.com/images/docs/trace-conciseness-score.jpg)
_包含简洁性（conciseness）分数的示例 Trace_
